In [30]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [32]:
df_8 = pd.read_excel("tables/Отчет_по_месяцам_8А_2023_2024.xlsx", header=1)
df_9 = pd.read_excel("tables/Отчет_по_месяцам_9А_2024_2025.xlsx", header=1)
df_10 = pd.read_excel("tables/Отчет_по_месяцам_10А_2023_2024.xlsx", header=1)
df_11 = pd.read_excel("tables/Отчет_по_месяцам_11А_2024_2025.xlsx", header=1)
df_8.head(15)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 244,Unnamed: 245,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253
0,Отчет об успеваемости и посещаемости ученика,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Учебный год: 2023/2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Класс: 8А,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Период: с 01.09.23 по 31.05.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Ученик 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Предмет,Сентябрь,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,1,4,5,7,8,11,12,13.0,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
def clean_df(df):
    words_to_exclude = ['Год', 'Класс', 'Период', 'год', 'Уважительная', 'Болезнь', 'Неуважительная', 'Опоздание']
    exclude_pattern = '|'.join(words_to_exclude)

    df_cleaned = df[~df.iloc[:, 0].str.contains(exclude_pattern, na=False)]
    df_cleaned = df_cleaned[~df_cleaned.iloc[:, 1].str.contains(exclude_pattern, na=False)]
    df_cleaned = df_cleaned.dropna(how='all')
    df_cleaned = df_cleaned.iloc[1:].reset_index(drop=True)

    return df_cleaned

In [34]:
def create_students_dict(df, rows_per_student):
    student_data = {}
    # Итерируемся по датасету с шагом в 'rows_per_student'
    for i in range(0, len(df), rows_per_student):
        student_df_full = df.iloc[i:i + rows_per_student].copy().reset_index(drop=True)

        # Получаем имя ученика из первой строки
        student_name = student_df_full.iloc[0, 0]  # Предполагаем, что имя ученика в первом столбце

        # Устанавливаем корректные заголовки
        header_subject = student_df_full.iloc[1].fillna('')
        header_date = student_df_full.iloc[2].fillna('')

        new_columns = []
        month = ''
        for j in range(len(header_subject)):
            if header_subject[j] and header_date[j]:
                new_columns.append(f"{header_subject[j]}")
                month = header_subject[j]
            elif header_subject[j]:
                new_columns.append(header_subject[j])
            else:
                str_to_append = f"{month}_{header_date[j]}"
                if str_to_append in new_columns:
                    new_columns.append(f"{month}_{header_date[j]}.1")
                else:
                    new_columns.append(f"{month}_{header_date[j]}")

        student_df_full.columns = new_columns
        student_df_processed = student_df_full.iloc[3:].reset_index(drop=True)
        student_df_processed.rename(columns={student_df_processed.columns[0]: 'Предмет'}, inplace=True)

        # Ищем индекс столбца "Средняя оценка"
        try:
            last_valid_column_index = student_df_processed.columns.get_loc('Средняя оценка') + 1
            # Выбираем все столбцы до "Средняя оценка" включительно
            cleaned_student_df = student_df_processed.iloc[:, :last_valid_column_index].copy()
            student_data[student_name] = cleaned_student_df
        except KeyError:
            print(f"В данных ученика '{student_name}' не найден столбец 'Средняя оценка'.")
            student_data[student_name] = student_df_processed.copy() # Сохраняем весь DataFrame, если столбец не найден

    return student_data

In [35]:
class StudentAnalyzer:
    def __init__(self, student_df):
        self.student_df = student_df
        self.months = ['Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь', 'Январь', 'Февраль', 'Март', 'Апрель', 'Май']
        self.monthly_grades = {month: [] for month in self.months}
        self.non_numeric_monthly = {month: [] for month in self.months}
        self.metrics = {}
        self._extract_month_data()

    @staticmethod
    def clean_grade(grade):
        if isinstance(grade, str):
            if '/' in grade:
                try:
                    return np.mean([float(x) for x in grade.split('/')])
                except ValueError:
                    return np.nan
            elif grade.strip().lower() in ['у', 'б', 'н']:
                return grade.strip().lower()
        try:
            return float(grade)
        except (ValueError, TypeError):
            return np.nan

    @staticmethod
    def calculate_grade_counts(grades_list):
        return {f'кол-во {int(g)}': grades_list.count(g) for g in [5.0, 4.0, 3.0, 2.0]}

    @staticmethod
    def calculate_status_counts(statuses_list):
        return {f'кол-во {s.upper()}': statuses_list.count(s) for s in ['у', 'б', 'н']}

    def _extract_month_data(self):
        month_start_indices = {col: i for i, col in enumerate(self.student_df.columns) if col in self.months}
        month_keys = list(month_start_indices.keys())

        month_column_ranges = {}
        for i, month in enumerate(month_keys):
            start = month_start_indices[month]
            if i + 1 < len(month_keys):
                end = month_start_indices[month_keys[i + 1]]
            elif 'Средняя оценка' in self.student_df.columns:
                end = self.student_df.columns.get_loc('Средняя оценка')
            else:
                end = len(self.student_df.columns)
            month_column_ranges[month] = list(range(start, end))

        for month, indices in month_column_ranges.items():
            for col in [self.student_df.columns[i] for i in indices]:
                cleaned = self.student_df[col].apply(self.clean_grade).dropna()
                numeric = pd.to_numeric(cleaned, errors='coerce').dropna()
                if not numeric.empty:
                    self.monthly_grades[month].extend(numeric.tolist())
                non_numeric = cleaned[~pd.to_numeric(cleaned, errors='coerce').notna()]
                if not non_numeric.empty:
                    self.non_numeric_monthly[month].extend(non_numeric.tolist())

    def analyze(self):
        self._calculate_monthly_averages()
        self._calculate_three_month_averages()
        self._calculate_year_and_semesters_stats()
        self._calculate_monthly_averages_counts()
        self._determine_success()
        return self.metrics

    def _calculate_monthly_averages(self):
        for month in self.months:
            grades = self.monthly_grades[month]
            self.metrics[f'Средний балл за {month}'] = np.mean(grades) if grades else np.nan

    def _calculate_three_month_averages(self):
        blocks = [['Сентябрь', 'Октябрь', 'Ноябрь'], ['Декабрь', 'Январь', 'Февраль'], ['Март', 'Апрель', 'Май']]
        for block in blocks:
            avgs = [self.metrics[f'Средний балл за {m}'] for m in block if pd.notna(self.metrics[f'Средний балл за {m}'])]
            self.metrics[f'Средний балл за 3 месяца ({block[0]}-{block[1]}-{block[2]})'] = np.mean(avgs) if avgs else np.nan

    def _calculate_year_and_semesters_stats(self):
        all_grades = self._get_combined_grades(self.months)
        all_statuses = self._get_combined_statuses(self.months)

        for key, val in self.calculate_grade_counts(all_grades).items():
            self.metrics[f'Общее {key} за год'] = val
        for key, val in self.calculate_status_counts(all_statuses).items():
            self.metrics[f'Общее {key} за год'] = val

        self._semester_stats('1 полугодие', ['Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь'])
        self._semester_stats('2 полугодие', ['Январь', 'Февраль', 'Март', 'Апрель', 'Май'])

    def _semester_stats(self, label, months):
        grades = self._get_combined_grades(months)
        statuses = self._get_combined_statuses(months)
        row_count = len(self.student_df)

        for key, val in self.calculate_grade_counts(grades).items():
            self.metrics[f'Общее {key} за {label}'] = val
        for key, val in self.calculate_status_counts(statuses).items():
            self.metrics[f'Общее {key} за {label}'] = val

    def _calculate_monthly_averages_counts(self):
        valid_months_grades = [m for m in self.months if self.monthly_grades[m]]
        valid_months_statuses = [m for m in self.months if self.non_numeric_monthly[m]]

        # for grade in [5.0, 4.0, 3.0, 2.0]:
        #     total = sum([self.monthly_grades[m].count(grade) for m in valid_months_grades])
        #     self.metrics[f'Среднее кол-во {int(grade)} в месяц'] = total / len(valid_months_grades) if valid_months_grades else np.nan

        for status in ['у', 'б', 'н']:
            total = sum([self.non_numeric_monthly[m].count(status) for m in valid_months_statuses])
            self.metrics[f'Среднее кол-во {status.upper()} в месяц'] = total / len(valid_months_statuses) if valid_months_statuses else np.nan

    def _determine_success(self):
        """
        Успешный студент — если по каждому предмету (строке) средний балл >= 3.5
        """
        all_columns = []
        for month in self.months:
            all_columns.extend([col for col in self.student_df.columns if col.startswith(month)])
        
        for _, row in self.student_df.iterrows():
            grades = []
            for col in all_columns:
                if col in row:
                    val = StudentAnalyzer.clean_grade(row[col])
                    if isinstance(val, float) and pd.notna(val):
                        grades.append(val)
            if grades:
                avg = np.mean(grades)
                if avg < 3.5:
                    self.metrics['Успешный (нет предмета со ср. баллом < 3.5)'] = 0
                    return
        self.metrics['Успешный (нет предмета со ср. баллом < 3.5)'] = 1

    def _get_combined_grades(self, months):
        grades = []
        for m in months:
            grades.extend(self.monthly_grades[m])
        return [g for g in grades if pd.notna(g)]

    def _get_combined_statuses(self, months):
        statuses = []
        for m in months:
            statuses.extend(self.non_numeric_monthly[m])
        return statuses


In [94]:
def create_dataset_for_clast(student_dict, class_name=''):
    all_students_metrics = []
    for student_name, student_df in student_dict.items():
        analyzer = StudentAnalyzer(student_df)
        metrics = analyzer.analyze()
        metrics['Ученик'] = f"{student_name}_{class_name}"
        all_students_metrics.append(metrics)
    final_metrics_df = pd.DataFrame(all_students_metrics)
    
    column_list = final_metrics_df.columns.tolist()
    last_column = column_list[-1]
    other_columns = column_list[:-1]
    new_column_order = [last_column] + other_columns
    final_metrics_df = final_metrics_df[new_column_order]
    
    return final_metrics_df

In [96]:
def del_drop_stud(df):
    first_column_name = df.columns[0]
    mask = ~df[first_column_name].str.contains('выб', na=False)
    df = df[mask]
    return df

In [98]:
df_8_clean = clean_df(df_8.copy())
df_9_clean = clean_df(df_9.copy())
df_10_clean = clean_df(df_10.copy())
df_11_clean = clean_df(df_11.copy())
student_dict_8 = create_students_dict(df_8_clean, 24)
student_dict_9 = create_students_dict(df_9_clean, 20)
student_dict_10 = create_students_dict(df_10_clean, 24)
student_dict_11 = create_students_dict(df_11_clean, 22)

In [108]:
final_metrics_df_8 = create_dataset_for_clast(student_dict_8, '8A')
final_metrics_df_9 = create_dataset_for_clast(student_dict_9, '9A')
final_metrics_df_10 = create_dataset_for_clast(student_dict_10, '10A')
final_metrics_df_11 = create_dataset_for_clast(student_dict_11, '11A')

In [109]:
try:
    final_metrics_df_9 = final_metrics_df_9.drop(columns=['Средний балл за Май'])
    final_metrics_df_11 = final_metrics_df_11.drop(columns=['Средний балл за Май'])
except:
    pass

In [102]:
final_metrics_df_8.to_csv('datasets/8A_для_кластеризации.csv',index=False, encoding="utf-8-sig")
final_metrics_df_9.to_csv('datasets/9A_для_кластеризации.csv',index=False, encoding="utf-8-sig")
final_metrics_df_10.to_csv('datasets/10A_для_кластеризации.csv',index=False, encoding="utf-8-sig")
final_metrics_df_11.to_csv('datasets/11A_для_кластеризации.csv',index=False, encoding="utf-8-sig")

In [112]:
summary_final_metrics_df = pd.concat([final_metrics_df_8, 
                                      final_metrics_df_9, 
                                      final_metrics_df_10, 
                                      final_metrics_df_11], 
                                     ignore_index=True)
summary_final_metrics_df

,Ученик,Средний балл за Сентябрь,Средний балл за Октябрь,Средний балл за Ноябрь,Средний балл за Декабрь,Средний балл за Январь,Средний балл за Февраль,Средний балл за Март,Средний балл за Апрель,Средний балл за Май,...,Общее кол-во 4 за 2 полугодие,Общее кол-во 3 за 2 полугодие,Общее кол-во 2 за 2 полугодие,Общее кол-во У за 2 полугодие,Общее кол-во Б за 2 полугодие,Общее кол-во Н за 2 полугодие,Среднее кол-во У в месяц,Среднее кол-во Б в месяц,Среднее кол-во Н в месяц,Успешный (нет предмета со ср. баллом < 3.5)
0,Ученик 1_8A,3.807018,3.600000,3.852941,3.666667,3.902439,3.839286,3.800000,3.830000,3.860000,...,55,61,7,0,126,21,2.111111,20.333333,2.555556,0
1,Ученик 2_8A,3.382979,3.212121,3.483871,3.316667,3.342857,3.300000,3.150000,3.261905,3.304878,...,30,115,26,11,64,25,4.888889,13.777778,2.888889,0
2,Ученик выбывший_8A,3.342105,3.378049,3.289474,3.370968,3.294118,3.531250,3.711538,3.387097,3.000000,...,32,47,15,237,35,25,37.875000,7.750000,3.125000,0
3,Ученик 3_8A,2.965517,2.972222,3.085366,3.166667,3.150000,3.048077,3.089744,3.108696,3.317073,...,32,132,48,7,0,8,3.250000,7.250000,1.125000,0
4,Ученица 4_8A,3.842593,3.702703,3.646341,3.576923,3.566667,3.518182,3.750000,3.525000,4.015152,...,54,85,18,39,0,8,11.500000,3.375000,1.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Ученица 19_11A,4.108108,3.967391,4.135135,4.138889,4.142857,4.200000,4.120000,3.909091,NaN,...,32,25,2,6,0,0,5.333333,0.000000,0.666667,0
94,Ученица 20_11A,3.440000,3.708333,3.595238,3.937500,3.675676,3.800000,3.605263,3.750000,NaN,...,33,27,13,0,22,0,0.333333,11.333333,0.333333,0
95,Ученик 21_11A,4.111111,3.532609,3.627907,3.925926,3.680000,3.812500,3.812500,2.000000,NaN,...,20,12,10,6,143,4,0.875000,23.375000,0.750000,0
96,Ученица 22_11A,4.160000,4.029412,4.035714,4.166667,4.090909,4.260870,4.384615,3.958333,NaN,...,41,16,0,19,3,0,5.200000,6.400000,0.400000,1


In [114]:
summary_final_metrics_df = del_drop_stud(summary_final_metrics_df)

In [124]:
mean_value = summary_final_metrics_df['Средний балл за Май'].mean()
summary_final_metrics_df['Средний балл за Май'].fillna(mean_value, inplace=True)

In [128]:
summary_final_metrics_df.to_csv('datasets/суммарный_для_кластеризации.csv',index=False, encoding="utf-8-sig")